In [1]:
import os
import hail as hl

In [2]:
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://c14n02.ruddle.hpc.yale.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.58-3f304aae6ce2
LOGGING: writing to /gpfs/ycga/project/kahle/sp2349/weilai_gnomad/code/hail-20210218-1658-0.2.58-3f304aae6ce2.log


In [3]:
#Load Combined sites ch5 mt table (converted from weilai exome and genome vcfs). Exome and Genome stats are separated.
combined = hl.read_table('/gpfs/ycga/project/kahle/sp2349/weilai_gnomad/tables/combined.gnomad.r2.1.1.sites.5.raw.mt')
#Mt table with combined results from exome and genome. Row names differ from combined mt
converted = hl.read_table('/gpfs/ycga/project/kahle/sp2349/weilai_gnomad/tables/combined.gnomad.r2.1.1.sites.5.mt')

#Load gnomad exome table
#gnomad_e = hl.read_table('/gpfs/ycga/project/kahle/sp2349/weilai_gnomad/tables/gnomad_e.diaph1.coords.mt')

gnomad_g = hl.read_matrix_table('/gpfs/ycga/project/kahle/sp2349/weilai_gnomad/tables/gnomad.genomes.r2.1.1.sites.5.decomposed.normalized.hg19_multianno.hg19_multianno.mt')
gnomad_e = hl.read_matrix_table('/gpfs/ycga/project/kahle/sp2349/weilai_gnomad/tables/gnomad.exomes.r2.1.1.sites.5.decomposed.normalized_anno.hg19_multianno.hg19_multianno.hg19_multianno.hg19_multianno.hg19_multianno.mt')

In [5]:
vim 

KeyboardInterrupt: 

In [ ]:
gnomad_g.rows().show()

In [ ]:
gnomad_e = gnomad_e.rows()
gnomad_e.filter(gnomad_e.filters.contains("PASS")).show()

In [ ]:
hl.is_missing(combined.filters).show()

In [ ]:
combined.filters_1.show()

In [ ]:
combined.aggregate(hl.agg.counter(combined.filters))

In [ ]:
combined.aggregate(hl.agg.counter(combined.info_1.filters))

In [ ]:
combined.filter(combined.locus.position == 140896422).filters.show()

In [ ]:
combined.filter(combined.locus.position == 140896422).filters_1.show()

In [ ]:
combined.filter(combined.locus.position == 140896422).info_1.CADD16snv_PHRED[0].show()

In [ ]:
hl.float64(combined.filter(combined.locus.position == 140896422).info_1.CADD16snv_PHRED[0]).show()

In [ ]:
hl.is_missing(combined.filter(combined.locus.position == 140896422).info.CADD16snv_PHRED[0]).show()

In [ ]:
check.info.CADD16snv_PHRED.is_missing()

In [ ]:
test = combined.filter(combined.info.CADD16snv_PHRED[0] == 'NA')
test.drop(test.info_1)
test.info.CADD16snv_PHRED.show()

In [ ]:
converted.filter(converted.locus.position == 140896422).show()

In [ ]:
converted = None

In [ ]:
#Make NA values 0 (float) if position missing in exomes
converted = combined.annotate(info=combined.info.annotate(non_topmed_AC=hl.if_else(hl.is_missing(combined.info.non_topmed_AC) == True,0,combined.info.non_topmed_AC)))
converted = converted.annotate(info=converted.info.annotate(non_topmed_AN=hl.if_else(hl.is_missing(converted.info.non_topmed_AN) == True,0,converted.info.non_topmed_AN)))
converted = converted.annotate(info=converted.info.annotate(non_topmed_nhomalt=hl.if_else(hl.is_missing(converted.info.non_topmed_nhomalt) == True,0,converted.info.non_topmed_nhomalt)))

In [ ]:
converted = converted.annotate(combined_nontopmed_AC=converted.info.non_topmed_AC + converted.info_1.non_topmed_AC)
converted = converted.annotate(combined_nontopmed_AN=converted.info.non_topmed_AN + converted.info_1.non_topmed_AN)
converted = converted.annotate(combined_nontopmed_nhomalt=converted.info.non_topmed_nhomalt + converted.info_1.non_topmed_nhomalt)


#Remove any 'NA' values in annovar fields
#converted = converted.annotate(info=converted.info.annotate(CADD_phred=hl.if_else(converted.info.CADD_phred == 'NA', hl.null('float64'), converted.info.CADD_phred)))
#converted = converted.annotate(info_1=converted.info_1.annotate(CADD_phred=hl.if_else(converted.info_1.CADD_phred == 'NA', hl.null('float64'), converted.info_1.CADD_phred)))
converted = converted.annotate(info=converted.info.annotate(CADD16snv_PHRED=hl.if_else(converted.info.CADD16snv_PHRED[0] == 'NA', hl.null('array<str>'), converted.info.CADD16snv_PHRED)))
converted = converted.annotate(info_1=converted.info_1.annotate(CADD16snv_PHRED=hl.if_else(converted.info_1.CADD16snv_PHRED[0] == 'NA', hl.null('array<str>'), converted.info_1.CADD16snv_PHRED)))
converted = converted.annotate(info=converted.info.annotate(CADD16indel_PHRED=hl.if_else(converted.info.CADD16indel_PHRED[0] == 'NA', hl.null('array<str>'), converted.info.CADD16indel_PHRED)))
converted = converted.annotate(info_1=converted.info_1.annotate(CADD16indel_PHRED=hl.if_else(converted.info_1.CADD16indel_PHRED[0] == 'NA', hl.null('array<str>'), converted.info_1.CADD16indel_PHRED)))
converted = converted.annotate(info=converted.info.annotate(MPC=hl.if_else(converted.info.MPC[0] == 'NA', hl.null('array<str>'), converted.info.MPC)))
converted = converted.annotate(info_1=converted.info_1.annotate(MPC=hl.if_else(converted.info_1.MPC[0] == 'NA', hl.null('array<str>'), converted.info_1.MPC)))

In [ ]:
converted = converted.transmute(exome_filters=converted.filters)
converted = converted.transmute(genome_filters=converted.filters_1)
converted = converted.transmute(exome_qual = converted.qual)
converted = converted.transmute(genome_qual = converted.qual_1)
converted = converted.transmute(exome_rsid = converted.rsid)
converted = converted.transmute(genome_rsid = converted.rsid_1) 

In [ ]:
combined.

In [ ]:
converted.filter(converted.locus.position == 140896422).info_1.CADD16snv_PHRED[0].show()

In [ ]:
hl.float64(converted.filter(converted.locus.position == 140896422).info_1.CADD16snv_PHRED[0]).show()

In [ ]:
gnomad_e.filter(gnomad_e.locus.position == 140896422).show()

In [ ]:
gnomad_g.filter(gnomad_g.locus.position == 140896422).show()

In [ ]:
test = combined.annotate(CADD_phred=hl.if_else(hl.is_missing(combined.info.CADD_phred) == True, hl.if_else(hl.is_missing(combined.info_1.CADD_phred) == True, hl.null('float64'), combined.info_1.CADD_phred), combined.info.CADD_phred))

In [ ]:
test = combined.annotate(CADD16snv_PHRED=hl.if_else(combined.info.CADD16snv_PHRED[0] == 'NA', hl.if_else(combined.info_1.CADD16snv_PHRED[0] == 'NA', hl.null(hl.tfloat),hl.float64(combined.info_1.CADD16snv_PHRED[0])),hl.float64(combined.info.CADD16snv_PHRED[0])))

In [ ]:
hl.if_else(hl.is_missing(combined.info.CADD16snv_PHRED) == True, hl.if_else(hl.is_missing(combined.info_1.CADD16snv_PHRED) == True, hl.null(hl.tfloat64),hl.float64(combined.info_1.CADD16snv_PHRED[0])),hl.float64(combined.info.CADD16snv_PHRED[0])).show()

In [ ]:
converted = converted.annotate(CADD16indel_PHRED=hl.if_else(hl.is_missing(converted.info.CADD16indel_PHRED) == True, hl.if_else(hl.is_missing(converted.info_1.CADD16indel_PHRED) == True, hl.null('float64'), hl.float64(converted.info_1.CADD16indel_PHRED[0])), hl.float64(converted.info.CADD16indel_PHRED[0])))

In [ ]:
hl.if_else(hl.is_missing(combined.info.CADD16indel_PHRED) == True, hl.if_else(hl.is_missing(combined.info_1.CADD16indel_PHRED) == True, hl.null('float64'), hl.float64(combined.info_1.CADD16indel_PHRED[0])), hl.float64(combined.info.CADD16indel_PHRED[0]))

In [ ]:
hl.if_else(hl.is_missing(converted.info.MPC) == True, hl.if_else(hl.is_missing(converted.info_1.MPC) == True, hl.null('float64'), hl.float64(converted.info_1.MPC[0])),hl.float64(converted.info.MPC[0]))
hl.if_else(hl.is_missing(combined.info.MPC) == True, hl.if_else(hl.is_missing(combined.info_1.MPC) == True, hl.null('float64'), hl.float64(combined.info_1.MPC[0])),hl.float64(combined.info.MPC[0]))

In [ ]:
hl.if_else(hl.is_missing(combined.info["Func.refGene"][0]) == True, hl.if_else(hl.is_missing(combined.info_1["Func.refGene"][0]) == True, hl.null('str'),combined.info_1["Func.refGene"][0]), combined.info["Func.refGene"][0])